In [ ]:
library(rvest)
library(magrittr)

getistp <- function(url){
  dat <- read_html(url)
  XPath1 <- "//td[2]/a"					# 「科目名」部分のXPath　※幾つかの科目で調べて
  subj <- html_nodes(dat, xpath=XPath1)				# <a>タグの内容

  sn <- html_text(subj)					# 科目名（<a>タグのテキスト）			

  spage <- html_attr(subj,"href")				# 科目シラバスのURLの一部（<a>タグのhref属性）
  "-------------------------------------------------------"
  id <-gsub("^.*kougicd)=(\\d+).*", "\\1",  spage);     	# spageから科目コード取得
  id <- as.numeric(id)	        # 数値に変換
  "-------------------------------------------------------"
  XPath2 <- "//td[4]"			# 「担当教員」部分のXPath
  teach <- html_nodes(dat, xpath=XPath2)		# タグの内容
  "-------------------------------------------------------"
  tn <- html_text(teach); tn[1:3]		# 教員名（タグのテキスト）
  tn <- tn[-1]; tn[1:3]			# 不適切な1番目の要素を除く
  # 複数の教員が担当する授業では，一つに結合した教員名となる
  "-------------------------------------------------------"
  tbl <- data.frame(id,sn,tn,spage); 	# 科目一覧（「id，科目名，教員名, 科目URL」の表）
  return(tbl)
}

getistp2 <- function(urls){
  if(length(urls)==1){  #変数urlsに含まれる文字列が1つの時
    res <- getistp(urls)  #関数getistpへurlsをurlとして扱いそのまま実行
  }else{
    res <- c()  #resの空ベクターを生成
    for(u in urls){
      res <- rbind(res, getistp(u)) #getistpをurlsに格納されている文字列の中から1つずつ順番に実行し、できたdataframe型を変数resで結合する。
    }
  }
  return(res) #結合されたresが返される。
}

#−−−−−−−−−−−−−−−−−−−−−−−−−−−−−−−−−−−−−−−−
# 使用例 2022年度カリキュラム
urls <- c("/subject1.html", "/subject2.html") #file名を単数形に変更した
tbl <- getistp2(urls)
write.csv(tbl, file="subj_NE2022.csv", row.names=FALSE)
# 新規ファイルが見えない場合，セッションストレージの「更新」アイコンをクリック